In [1]:
class base:
    """
    Base class representing an entity with a name.

    Attributes:
        name (str): The name of the entity.
    """

    def __init__(self, name: str):
        """
        Initialize a base instance with a given name.

        Args:
            name (str): Name of the entity.
        """
        self.name = name.format()

    def say_hello(self):
        """
        Print a greeting message addressing the entity by its name.
        """
        print(f"Hello, {self.name}!")

class derived(base):
    """
    Derived class inheriting from 'base', representing a specialized entity.
    """

    def __init__(self, name: str):
        """
        Initialize a derived instance with a given name.
        
        Args:
            name (str): Name of the entity.
        """
        super().__init__(name)

    def say_hello(self):
        """
        Print a greeting message addressing the entity by its name.
        Overrides the base class implementation.
        """
        print(f"Hello, {self.name}!")

def main():
    b = base("base")
    d = derived("derived")
    b.say_hello()
    d.say_hello()

if __name__ == "__main__":
    main()



Hello, base!
Hello, derived!
